In [7]:
import torch
import glob
from PIL import Image
from clip.clip import load
from concept_extraction.concept_extraction import prepare_image_from_datasets, extract_embedding

folder_path = "stl10_sample_500"
ViT_B_16_clip, image_transform = load("ViT-B/16")

image_paths = glob.glob(folder_path + "/*.png")
samples = []
for img_path in image_paths:
    img = Image.open(img_path).convert("RGB")
    samples.append({"image": img})

embeddings_list = []
for sample in samples:
    _, image_transformed = prepare_image_from_datasets(sample["image"], image_transform)
    embedding = extract_embedding(image_transformed=image_transformed, clip_model=ViT_B_16_clip)
    embeddings_list.append(embedding)

embeddings = torch.stack(embeddings_list)

torch.save({
    "embeddings": embeddings,
    "image_paths": image_paths,
}, "precomputed_embeddings.pt")

print(f"Saved {len(samples)} embeddings to precomputed_embeddings.pt")


ModuleNotFoundError: No module named 'jaxtyping'

In [ ]:
import torch
import glob
from PIL import Image
from clip.clip import load
from concept_extraction.concept_extraction import prepare_image_from_datasets, extract_embedding

In [ ]:
folder_path = "stl10_sample_500"
ViT_B_16_clip, image_transform = load("ViT-B/16")

image_paths = glob.glob(folder_path + "/*.png")
samples = []

In [30]:
for img_path in image_paths:
    img = Image.open(img_path).convert("RGB")
    samples.append({"image": img})

In [32]:
import os
print(os.getcwd())  # aktualny katalog roboczy
print(os.listdir("stl10_sample_500"))  # co jest w folderze?


/Users/zuzannasienko/Documents/github/spire


FileNotFoundError: [Errno 2] No such file or directory: 'stl10_sample_500'

In [31]:
samples

[]

In [ ]:
for img_path in image_paths:
    img = Image.open(img_path).convert("RGB")
    samples.append({"image": img})

embeddings_list = []
for sample in samples:
    _, image_transformed = prepare_image_from_datasets(sample["image"], image_transform)
    embedding = extract_embedding(image_transformed=image_transformed, clip_model=ViT_B_16_clip)
    embeddings_list.append(embedding)

embeddings = torch.stack(embeddings_list)

torch.save({
    "embeddings": embeddings,
    "image_paths": image_paths,
}, "precomputed_embeddings.pt")

print(f"Saved {len(samples)} embeddings to precomputed_embeddings.pt")


RuntimeError: stack expects a non-empty TensorList

In [24]:
embeddings_list = []
for i, sample in enumerate(samples):
    _, image_transformed = prepare_image_from_datasets(sample["image"], image_transform)
    embedding = extract_embedding(image_transformed=image_transformed, clip_model=ViT_B_16_clip)
    print(f"Embedding {i}: type={type(embedding)}, shape={embedding.shape if embedding is not None else None}")
    embeddings_list.append(embedding)


In [26]:
_, image_transformed = prepare_image_from_datasets(sample["image"], image_transform)
print(type(image_transformed), image_transformed.shape)


NameError: name 'sample' is not defined